# Программирование на Python
## Продвинутые методы работы с данными в `pandas`
### Вспомогательные материалы
1. л

### Ответы на вопросы (их нет)

In [ ]:
# наш код здесь

### А теперь к сегодняшней теме
Сегодня мы будем работать с данными по COVID-19 (взяты [отсюда](https://github.com/datasets/covid-19)). Датасет содержит следующие переменные:

- `Date` - дата;
- `Country/Region` - страна/регион;
- `Province/State` - штат/провинция;
- `Confirmed` - количество подтвержденных случаев заражения (на указанную дату);
- `Recovered` - количество выздоровевших (на указанную дату);
- `Deaths` - количество умерших (на указанную дату).

Импортируйте и проведите первичное изучение данных (соответствующим типу каждой переменной). Исследуем данные целиком:

In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv('time-series-19-covid-combined.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231744 entries, 0 to 231743
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Date            231744 non-null  object 
 1   Country/Region  231744 non-null  object 
 2   Province/State  72624 non-null   object 
 3   Confirmed       231744 non-null  int64  
 4   Recovered       218688 non-null  float64
 5   Deaths          231744 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 10.6+ MB


In [2]:
#записки:

#1 есть пропущенные значения: Province/State, Recovered
#2 лучше привести названия колонок к стандартному виду (как переменные в Python)
#3 колонка Recovered имеет нецелый тип, хотя это количество -> из-за np.nan

In [3]:
data.head()

Date Country/Region Province/State  Confirmed  Recovered  Deaths
0  2020-01-22    Afghanistan            NaN          0        0.0       0
1  2020-01-23    Afghanistan            NaN          0        0.0       0
2  2020-01-24    Afghanistan            NaN          0        0.0       0
3  2020-01-25    Afghanistan            NaN          0        0.0       0
4  2020-01-26    Afghanistan            NaN          0        0.0       0

In [4]:
data.Date

0         2020-01-22
1         2020-01-23
2         2020-01-24
3         2020-01-25
4         2020-01-26
             ...    
231739    2022-04-12
231740    2022-04-13
231741    2022-04-14
231742    2022-04-15
231743    2022-04-16
Name: Date, Length: 231744, dtype: object

In [5]:
np.array([np.nan, 1, 2, 3])

array([nan,  1.,  2.,  3.])

In [6]:
type(np.nan)

float

Исследуем каждую отдельную переменную. Существуют следующие виды переменных:

![](https://slideplayer.com/slide/4457934/14/images/13/Types+of+Variables+Examples%3A+Marital+Status+Political+Party+Eye+Color.jpg)

Переменную (ее распределение) следует изучать сообразно ее виду, используя соответствующую метрику. Для категориальной это частотная таблица, для числовой - меры центральной тенденции (главным образом среднее и медиана) и меры разброса (размах, стандартное отклонение).

Категориальные:

In [7]:
data.head()

Date Country/Region Province/State  Confirmed  Recovered  Deaths
0  2020-01-22    Afghanistan            NaN          0        0.0       0
1  2020-01-23    Afghanistan            NaN          0        0.0       0
2  2020-01-24    Afghanistan            NaN          0        0.0       0
3  2020-01-25    Afghanistan            NaN          0        0.0       0
4  2020-01-26    Afghanistan            NaN          0        0.0       0

In [8]:
data['Country/Region'].value_counts()

Country/Region
China             27744
Canada            13056
United Kingdom    11424
France             9792
Australia          6528
                  ...  
Guinea              816
Guinea-Bissau       816
Guyana              816
Haiti               816
Zimbabwe            816
Name: count, Length: 198, dtype: int64

In [9]:
data['Country/Region'].value_counts().iloc[:15]

Country/Region
China                    27744
Canada                   13056
United Kingdom           11424
France                    9792
Australia                 6528
Netherlands               4080
Denmark                   2448
New Zealand               1632
Portugal                   816
Qatar                      816
Niger                      816
Nigeria                    816
North Macedonia            816
Norway                     816
Saint Kitts and Nevis      816
Name: count, dtype: int64

Числовые переменные:

In [10]:
data.describe()

Confirmed     Recovered         Deaths
count  2.317440e+05  2.186880e+05  231744.000000
mean   5.132362e+05  1.062116e+05    9760.170231
std    3.002239e+06  8.397413e+05   49749.871513
min    0.000000e+00  0.000000e+00       0.000000
25%    2.590000e+02  0.000000e+00       2.000000
50%    5.725000e+03  5.500000e+01      73.000000
75%    1.095645e+05  5.002000e+03    1777.000000
max    8.062512e+07  3.097475e+07  988609.000000

По окончании изучения составьте список действий, которые необходимо произвести с данными перед дальнейшим анализом.

### Выполните предобработку данных
1. Стандартизируйте названия переменных;
2. Переведите переменную `Date` в тип `datetime`;
3. "Схлопните" данные по провинциям в данные по стране в целом;
4. Преобразуйте накопленное значение по стране на дату в значение прироста по стране на дату.

Названия переменных (пригодится метод `rename`):

In [11]:
data.head()

Date Country/Region Province/State  Confirmed  Recovered  Deaths
0  2020-01-22    Afghanistan            NaN          0        0.0       0
1  2020-01-23    Afghanistan            NaN          0        0.0       0
2  2020-01-24    Afghanistan            NaN          0        0.0       0
3  2020-01-25    Afghanistan            NaN          0        0.0       0
4  2020-01-26    Afghanistan            NaN          0        0.0       0

In [12]:
?pd.DataFrame.rename

Signature:
pd.DataFrame.rename(
    self,
    mapper: 'Renamer | None' = None,
    *,
    index: 'Renamer | None' = None,
    columns: 'Renamer | None' = None,
    axis: 'Axis | None' = None,
    copy: 'bool | None' = None,
    inplace: 'bool' = False,
    level: 'Level | None' = None,
    errors: 'IgnoreRaise' = 'ignore',
) -> 'DataFrame | None'
Docstring:
Rename columns or index labels.

Function / dict values must be unique (1-to-1). Labels not contained in
a dict / Series will be left as-is. Extra labels listed don't throw an
error.

See the :ref:`user guide <basics.rename>` for more.

Parameters
----------
mapper : dict-like or function
    Dict-like or function transformations to apply to
    that axis' values. Use either ``mapper`` and ``axis`` to
    specify the axis to target with ``mapper``, or ``index`` and
    ``columns``.
index : dict-like or function
    Alternative to specifying axis (``mapper, axis=0``
    is equivalent to ``index=mapper``).
columns : dict-like or funct

In [13]:
data.columns

Index(['Date', 'Country/Region', 'Province/State', 'Confirmed', 'Recovered',
       'Deaths'],
      dtype='object')

In [14]:
data_renamed = data.rename(columns=lambda name: name.replace('/', '_').lower())

`Date` в `datetime` (пригодится `pd.to_datetime` и [таблица](https://www.w3schools.com/python/python_datetime.asp) кодов):

In [15]:
data_renamed.head()

date country_region province_state  confirmed  recovered  deaths
0  2020-01-22    Afghanistan            NaN          0        0.0       0
1  2020-01-23    Afghanistan            NaN          0        0.0       0
2  2020-01-24    Afghanistan            NaN          0        0.0       0
3  2020-01-25    Afghanistan            NaN          0        0.0       0
4  2020-01-26    Afghanistan            NaN          0        0.0       0

In [16]:
data_renamed['date_dt'] = pd.to_datetime(data_renamed['date'], format='%Y-%m-%d')
data_renamed.head()

date country_region province_state  confirmed  recovered  deaths  \
0  2020-01-22    Afghanistan            NaN          0        0.0       0   
1  2020-01-23    Afghanistan            NaN          0        0.0       0   
2  2020-01-24    Afghanistan            NaN          0        0.0       0   
3  2020-01-25    Afghanistan            NaN          0        0.0       0   
4  2020-01-26    Afghanistan            NaN          0        0.0       0   

     date_dt  
0 2020-01-22  
1 2020-01-23  
2 2020-01-24  
3 2020-01-25  
4 2020-01-26

От провинций к странам: `groupby` -> `aggregate`:

In [17]:
data_renamed[data_renamed['country_region'] == 'China']

date country_region province_state  confirmed  recovered  deaths  \
48144  2020-01-22          China          Anhui          1        0.0       0   
48145  2020-01-23          China          Anhui          9        0.0       0   
48146  2020-01-24          China          Anhui         15        0.0       0   
48147  2020-01-25          China          Anhui         39        0.0       0   
48148  2020-01-26          China          Anhui         60        0.0       0   
...           ...            ...            ...        ...        ...     ...   
75883  2022-04-12          China       Zhejiang       2850        0.0       1   
75884  2022-04-13          China       Zhejiang       2859        0.0       1   
75885  2022-04-14          China       Zhejiang       2859        0.0       1   
75886  2022-04-15          China       Zhejiang       2886        0.0       1   
75887  2022-04-16          China       Zhejiang       2897        0.0       1   

         date_dt  
48144 2020-01-22  
48145 2020-01-23  
48146 2020-01-24  
48147 2020-01-25  
48148 2020-01-26  
...          ...  
75883 2022-04-12  
75884 2022-04-13  
75885 2022-04-14  
75886 2022-04-15  
75887 2022-04-16  

[27744 rows x 7 columns]

In [18]:
data_groups = [*data_renamed.groupby(['country_region'])]

In [19]:
data_groups[0]

(('Afghanistan',),
            date country_region province_state  confirmed  recovered  deaths  \
 0    2020-01-22    Afghanistan            NaN          0        0.0       0   
 1    2020-01-23    Afghanistan            NaN          0        0.0       0   
 2    2020-01-24    Afghanistan            NaN          0        0.0       0   
 3    2020-01-25    Afghanistan            NaN          0        0.0       0   
 4    2020-01-26    Afghanistan            NaN          0        0.0       0   
 ..          ...            ...            ...        ...        ...     ...   
 811  2022-04-12    Afghanistan            NaN     178257        0.0    7676   
 812  2022-04-13    Afghanistan            NaN     178295        0.0    7676   
 813  2022-04-14    Afghanistan            NaN     178352        0.0    7676   
 814  2022-04-15    Afghanistan            NaN     178373        0.0    7676   
 815  2022-04-16    Afghanistan            NaN     178387        0.0    7676   
 
        date_dt  
 

In [20]:
data_groups[0][0]

('Afghanistan',)

In [21]:
data_groups[0][1]

date country_region province_state  confirmed  recovered  deaths  \
0    2020-01-22    Afghanistan            NaN          0        0.0       0   
1    2020-01-23    Afghanistan            NaN          0        0.0       0   
2    2020-01-24    Afghanistan            NaN          0        0.0       0   
3    2020-01-25    Afghanistan            NaN          0        0.0       0   
4    2020-01-26    Afghanistan            NaN          0        0.0       0   
..          ...            ...            ...        ...        ...     ...   
811  2022-04-12    Afghanistan            NaN     178257        0.0    7676   
812  2022-04-13    Afghanistan            NaN     178295        0.0    7676   
813  2022-04-14    Afghanistan            NaN     178352        0.0    7676   
814  2022-04-15    Afghanistan            NaN     178373        0.0    7676   
815  2022-04-16    Afghanistan            NaN     178387        0.0    7676   

       date_dt  
0   2020-01-22  
1   2020-01-23  
2   2020-01-24  
3   2020-01-25  
4   2020-01-26  
..         ...  
811 2022-04-12  
812 2022-04-13  
813 2022-04-14  
814 2022-04-15  
815 2022-04-16  

[816 rows x 7 columns]

In [22]:
len(data_groups), data_renamed['country_region'].nunique()

(198, 198)

In [23]:
data_groups[1][0]

('Albania',)

In [24]:
numeric = ['confirmed', 'recovered', 'deaths']

data_renamed.groupby(['date', 'date_dt', 'country_region'], as_index=False)[numeric].agg('sum')

date    date_dt        country_region  confirmed  recovered  \
0       2020-01-22 2020-01-22           Afghanistan          0        0.0   
1       2020-01-22 2020-01-22               Albania          0        0.0   
2       2020-01-22 2020-01-22               Algeria          0        0.0   
3       2020-01-22 2020-01-22               Andorra          0        0.0   
4       2020-01-22 2020-01-22                Angola          0        0.0   
...            ...        ...                   ...        ...        ...   
161563  2022-04-16 2022-04-16    West Bank and Gaza     656617        0.0   
161564  2022-04-16 2022-04-16  Winter Olympics 2022        535        0.0   
161565  2022-04-16 2022-04-16                 Yemen      11817        0.0   
161566  2022-04-16 2022-04-16                Zambia     318467        0.0   
161567  2022-04-16 2022-04-16              Zimbabwe     247237        0.0   

        deaths  
0            0  
1            0  
2            0  
3            0  
4            0  
...        ...  
161563    5656  
161564       0  
161565    2148  
161566    3973  
161567    5462  

[161568 rows x 6 columns]

In [25]:
numeric = ['confirmed', 'recovered', 'deaths']

data_countries = data_renamed.groupby(['date', 'date_dt', 'country_region'], as_index=False)[numeric].sum()
data_countries['recovered'] = data_countries['recovered'].astype(int)
data_countries.head()

date    date_dt country_region  confirmed  recovered  deaths
0  2020-01-22 2020-01-22    Afghanistan          0          0       0
1  2020-01-22 2020-01-22        Albania          0          0       0
2  2020-01-22 2020-01-22        Algeria          0          0       0
3  2020-01-22 2020-01-22        Andorra          0          0       0
4  2020-01-22 2020-01-22         Angola          0          0       0

От накопленной суммы к приросту:

In [26]:
data_countries.head()

date    date_dt country_region  confirmed  recovered  deaths
0  2020-01-22 2020-01-22    Afghanistan          0          0       0
1  2020-01-22 2020-01-22        Albania          0          0       0
2  2020-01-22 2020-01-22        Algeria          0          0       0
3  2020-01-22 2020-01-22        Andorra          0          0       0
4  2020-01-22 2020-01-22         Angola          0          0       0

In [27]:
data_rus = data_countries[data_countries['country_region'] == 'Russia'].reset_index(drop=True)  # .copy()
data_rus.head()

date    date_dt country_region  confirmed  recovered  deaths
0  2020-01-22 2020-01-22         Russia          0          0       0
1  2020-01-23 2020-01-23         Russia          0          0       0
2  2020-01-24 2020-01-24         Russia          0          0       0
3  2020-01-25 2020-01-25         Russia          0          0       0
4  2020-01-26 2020-01-26         Russia          0          0       0

In [28]:
data_rus.tail()

date    date_dt country_region  confirmed  recovered  deaths
811  2022-04-12 2022-04-12         Russia   17756183          0  364779
812  2022-04-13 2022-04-13         Russia   17767760          0  365038
813  2022-04-14 2022-04-14         Russia   17778928          0  365285
814  2022-04-15 2022-04-15         Russia   17790211          0  365540
815  2022-04-16 2022-04-16         Russia   17801103          0  365774

In [29]:
data_rus['confirmed']

0             0
1             0
2             0
3             0
4             0
         ...   
811    17756183
812    17767760
813    17778928
814    17790211
815    17801103
Name: confirmed, Length: 816, dtype: int64

In [30]:
data_rus['confirmed'].shift(1)

0             NaN
1             0.0
2             0.0
3             0.0
4             0.0
          ...    
811    17745453.0
812    17756183.0
813    17767760.0
814    17778928.0
815    17790211.0
Name: confirmed, Length: 816, dtype: float64

In [31]:
data_rus['confirmed'].shift(1, fill_value=0)

0             0
1             0
2             0
3             0
4             0
         ...   
811    17745453
812    17756183
813    17767760
814    17778928
815    17790211
Name: confirmed, Length: 816, dtype: int64

In [32]:
data_rus['confirmed'] - data_rus['confirmed'].shift(1, fill_value=0)

0          0
1          0
2          0
3          0
4          0
       ...  
811    10730
812    11577
813    11168
814    11283
815    10892
Name: confirmed, Length: 816, dtype: int64

In [33]:
NUMERIC = ['confirmed', 'recovered', 'deaths']


def preprocess_country(country_df: pd.DataFrame) -> pd.DataFrame:
    """
    country_df: pd.DataFrame - dataframe for single country
    """
    country_df = country_df.copy()

    for column in NUMERIC:
        country_df[f'{column}_delta'] = country_df[column] - country_df[column].shift(1, fill_value=0)
        country_df.loc[country_df[f'{column}_delta'] < 0, f'{column}_delta'] = 0
        
    return country_df

In [34]:
?preprocess_country

Signature: preprocess_country(country_df: pandas.core.frame.DataFrame) -> pandas.core.frame.DataFrame
Docstring: country_df: pd.DataFrame - dataframe for single country
File:      /var/folders/h6/lv17v1r10lz21g745pd6774h0tgc4z/T/ipykernel_19681/271852743.py
Type:      function

In [35]:
preprocess_country(data_rus)

date    date_dt country_region  confirmed  recovered  deaths  \
0    2020-01-22 2020-01-22         Russia          0          0       0   
1    2020-01-23 2020-01-23         Russia          0          0       0   
2    2020-01-24 2020-01-24         Russia          0          0       0   
3    2020-01-25 2020-01-25         Russia          0          0       0   
4    2020-01-26 2020-01-26         Russia          0          0       0   
..          ...        ...            ...        ...        ...     ...   
811  2022-04-12 2022-04-12         Russia   17756183          0  364779   
812  2022-04-13 2022-04-13         Russia   17767760          0  365038   
813  2022-04-14 2022-04-14         Russia   17778928          0  365285   
814  2022-04-15 2022-04-15         Russia   17790211          0  365540   
815  2022-04-16 2022-04-16         Russia   17801103          0  365774   

     confirmed_delta  recovered_delta  deaths_delta  
0                  0                0             0  
1                  0                0             0  
2                  0                0             0  
3                  0                0             0  
4                  0                0             0  
..               ...              ...           ...  
811            10730                0           273  
812            11577                0           259  
813            11168                0           247  
814            11283                0           255  
815            10892                0           234  

[816 rows x 9 columns]

In [36]:
data_final = data_countries.groupby(['country_region'], as_index=False, group_keys=False).apply(preprocess_country)
data_final.head()

/var/folders/h6/lv17v1r10lz21g745pd6774h0tgc4z/T/ipykernel_19681/1354285442.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_final = data_countries.groupby(['country_region'], as_index=False, group_keys=False).apply(preprocess_country)


date    date_dt country_region  confirmed  recovered  deaths  \
0  2020-01-22 2020-01-22    Afghanistan          0          0       0   
1  2020-01-22 2020-01-22        Albania          0          0       0   
2  2020-01-22 2020-01-22        Algeria          0          0       0   
3  2020-01-22 2020-01-22        Andorra          0          0       0   
4  2020-01-22 2020-01-22         Angola          0          0       0   

   confirmed_delta  recovered_delta  deaths_delta  
0                0                0             0  
1                0                0             0  
2                0                0             0  
3                0                0             0  
4                0                0             0

In [37]:
np.all(data_final[['confirmed_delta', 'recovered_delta', 'deaths_delta']].values >= 0)

True

In [38]:
(data_final[['confirmed_delta', 'recovered_delta', 'deaths_delta']].values >= 0).mean()

1.0

#### Бойтесь типа `object`
На самом деле `np.ndarray` и `pd.Series` могут хранить объекты разных типов - с помощью типа `object`.

In [ ]:
weird_array = np.array([1, '1'], dtype='object')

Такой `array` лишается преимуществ массива. Пример:

In [ ]:
array_int = np.arange(100000)
array_object = array_int.astype('object')

In [ ]:
%%timeit

_ = array_int * 2

In [ ]:
%%timeit

_ = array_object * 2

Немного о `pd.concat` и возможностях оптимизации (а также опасностях `object`):

In [ ]:
numeric_cols = numeric + ['confirmed_delta', 'recovered_delta', 'deaths_delta']

In [ ]:
countries_dfs = [df[numeric_cols] for group, df in data_final.groupby(['country_region'])]

countries_dfs_concat = pd.concat(countries_dfs, ignore_index=True)

countries_dfs_concat.head()

In [ ]:
countries_nps = [df[numeric_cols].values for df in countries_dfs]

In [ ]:
countries_nps_concat = np.vstack(countries_nps)

In [ ]:
%%timeit

countries_dfs_concat = pd.concat(countries_dfs, ignore_index=True)

In [ ]:
%%timeit

countries_nps_concat = np.vstack(countries_nps)

**Важно!** Будьте осторожны с типами данных:)

#### pivot_table & melt
![](https://i.ytimg.com/vi/AsZPQyjYBhc/maxresdefault.jpg)

Как видно на картинке выше, табличные данные можно хранить в нескольких форматах.

In [ ]:
countries = ['US', 'Russia', 'United Kingdom']
columns_to_select = ['date_dt', 'country_region', 'confirmed_delta']

data_countries = data_final.loc[data_final['country_region'].isin(countries), columns_to_select].reset_index(drop=True)

data_countries.insert(0, 'year', data_countries['date_dt'].dt.year)

data_countries.head()

Переведем в wide формат:

In [ ]:
def extract_index(index_tuple: tuple) -> str:
    first, second = index_tuple

    if not second:
        return first
    
    return second

In [ ]:
data_countries_pivoted = data_countries.pivot(
    columns=['country_region'], index=['year', 'date_dt'], values=['confirmed_delta']
).reset_index()

data_countries_pivoted.columns = data_countries_pivoted.columns.map(extract_index)

data_countries_pivoted.head()

In [ ]:
data_countries.pivot_table(columns=['country_region'], index=['year', 'date_dt'], values=['confirmed_delta'])

Получите сводную таблицу, в которой:

- колонки - год и страны;
- в ячейках средние значения переменной `confirmed_delta` по году и стране.

In [ ]:
# наш код здесь

А теперь вернемся к long-формату:

In [ ]:
data_countries_pivoted.melt(
    id_vars=['year', 'date_dt'], value_vars=['Russia', 'US', 'United Kingdom'],
    var_name='country_region', value_name='confirmed_delta'
)

#### Дополняем данные с помощью другой таблицы
У вас есть другая таблица (`P_Popular Indicators.xlsx`), в которой находятся базовые индикаторы для некоторых стран. Нас интересуют три:

- Population, total - численность населения;
- GDP per capita (current US$) - ВВП на душу населения;
- Life expectancy at birth, total (years) - средняя ожидаемая продолжительность жизни.

Эти показатели пригодятся нам в дальнейшем исследовании. Присоедините их к главной таблице.

In [ ]:
indicators = ['Population, total', 'GDP per capita (current US$)', 'Life expectancy at birth, total (years)']

indicators_df = pd.read_excel('P_Popular Indicators.xlsx', ).drop(columns=['Series Code'])

indicators_df.info()

In [ ]:
# наш код здесь

#### Немного о визуализации
Изобразите распределение всех трех индикаторов с помощью:

1. Гистограммы;
2. Ящика с усами (boxplot).

На одном графике.

In [ ]:
# наш код здесь